<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/time_series_modeling_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 9th July, 2021
# Ref: https://github.com/philipperemy/keras-attention-mechanism
# Objective:
#            Use a multivariate time-series data to make prediction
#            for future.
#            

For example, given the stock prices data for last 10 days, as:<br> 
> <i>PrevClose,Open,High,Low,Last,VWAP,Volume_Turnover</i>, <b>Close</b> <br>
Make predictions of <b>Close</b> price for the next day. Here, we use all <u>eight</u> features of the past to make predictions for future.

## Install library

In [28]:
# 0.0 Install attention
! pip install attention

## Call libraries

In [29]:
# 1.0 Usual libraries
import numpy as np

# 1.1 tensorflow related libraries
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model
from attention import Attention

In [30]:
# 1.2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Generate some sample data
Data has just one feature time-value

In [31]:
# 2.0 Take past 10 time steps and make prediction for the next
#       time step
num_samples, time_steps, input_dim, output_dim = 100, 10, 8, 1
data_x = np.random.uniform(size=(num_samples, time_steps, input_dim))
data_y = np.random.uniform(size=(num_samples, output_dim))

In [32]:
# 2.1 Observe data
print(data_x.shape)
print(data_x[:2])   # Show 2 sets of 10 time-steps each


(100, 10, 8)
[[[7.02418148e-01 6.18682429e-01 2.24943995e-02 3.51913492e-01
   2.10419469e-01 4.32435525e-01 7.35911030e-01 1.12899552e-03]
  [9.45287137e-01 1.37788301e-01 1.10632684e-01 9.60056741e-01
   5.55394142e-01 8.93564152e-01 6.13409198e-01 9.11303900e-01]
  [2.89036619e-02 2.76187726e-02 8.74498411e-01 8.66868483e-01
   3.76526354e-01 7.80368995e-01 4.03277796e-01 2.90274774e-01]
  [3.20518444e-01 5.54589548e-02 8.83492761e-01 8.31341867e-01
   7.98950945e-01 9.10382844e-01 7.23219942e-01 7.43439220e-01]
  [7.23257934e-01 1.77441083e-01 8.60663525e-01 1.45703033e-02
   7.38449596e-01 6.37138469e-01 8.60187639e-01 2.34251128e-01]
  [5.75370300e-03 9.85433284e-02 9.94104370e-01 7.46644216e-01
   9.56902276e-01 3.73078193e-01 8.19067979e-01 5.94479013e-01]
  [2.09051774e-01 2.39954075e-02 7.52749014e-01 2.33582108e-01
   6.94264667e-02 5.78445506e-01 8.11222450e-01 5.33884378e-01]
  [8.19616666e-01 4.91226719e-01 4.79219635e-01 9.94039850e-01
   4.18924638e-01 9.68194654e-01 6.

In [33]:
# 2.2 Show next 4-values to be predicted
print(data_y[:4])   # Show 4 values

[[0.1525244 ]
 [0.64825146]
 [0.69913549]
 [0.96029106]]


### Our model

In [34]:
# 3.0 Define the model.
model_input = Input(shape=(time_steps, input_dim))
x = LSTM(64, return_sequences=True)(model_input)
x = Attention(32)(x)
x = Dense(1)(x)
model = Model(model_input, x)

In [35]:
# 3.1 Compile and print model summary
model.compile(loss='mae', optimizer='adam')
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 10, 8)]      0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 10, 64)       18688       input_3[0][0]                    
__________________________________________________________________________________________________
last_hidden_state (Lambda)      (None, 64)           0           lstm_2[0][0]                     
__________________________________________________________________________________________________
attention_score_vec (Dense)     (None, 10, 64)       4096        lstm_2[0][0]                     
____________________________________________________________________________________________

In [36]:
# 3.2 Start learning
model.fit(data_x, data_y, epochs=10)

Epoch 1/10
4/4 [==============================] - 2s 9ms/step - loss: 0.3425
Epoch 2/10
4/4 [==============================] - 0s 9ms/step - loss: 0.3361
Epoch 3/10
4/4 [==============================] - 0s 10ms/step - loss: 0.2528
Epoch 4/10
4/4 [==============================] - 0s 9ms/step - loss: 0.2729
Epoch 5/10
4/4 [==============================] - 0s 8ms/step - loss: 0.2649
Epoch 6/10
4/4 [==============================] - 0s 9ms/step - loss: 0.2498
Epoch 7/10
4/4 [==============================] - 0s 10ms/step - loss: 0.2508
Epoch 8/10
4/4 [==============================] - 0s 9ms/step - loss: 0.2476
Epoch 9/10
4/4 [==============================] - 0s 8ms/step - loss: 0.2547
Epoch 10/10
4/4 [==============================] - 0s 10ms/step - loss: 0.2550


In [37]:
# 3.3 Make predictions
pred1 = model.predict(data_x)

### Save model for future

In [38]:
# 4.0 Save model for future
model.save('test_model.h5')

In [39]:
# 4.1 Delete earlier model
del model

# 4.2 Load the saved model
model_h5 = load_model('test_model.h5')

In [40]:
# 4.3 Make predictions with the sabed model
pred2 = model_h5.predict(data_x)

In [41]:
# 4.4 Compare saved model predictions 
#      with predictions made before saving

np.testing.assert_almost_equal(pred1, pred2)
print('Success.')

Success.


In [ ]:
########## I am done ###############